# Finding Messi Goals

This notebook explores the statsbomb open data to extract data for all of Messi goals.

This will be use to schedule the tweets.

In [1]:
from geopy.geocoders import Nominatim
from timezonefinder import TimezoneFinder
import pytz
import pandas as pd
from statsbombpy import sb
from datetime import datetime

In [2]:
competitions = sb.competitions()

c:\Users\L781054\AppData\Local\anaconda3\envs\statsbomb\Lib\site-packages\statsbombpy\api_client.py:21: NoAuthWarning: credentials were not supplied. open data access only
  warnings.warn(


In [3]:
competitions['season_start'] = competitions.season_name.str.split("/",expand=True).iloc[:,0].astype(int)

In [8]:
competitions.country_name.value_counts()

country_name
Europe                       22
Spain                        21
International                11
England                       5
France                        3
Italy                         2
United States of America      2
Germany                       2
Argentina                     2
South America                 1
Africa                        1
India                         1
North and Central America     1
Name: count, dtype: int64

In [9]:
MESSI_TEAMS = {
    "Barcelona": [("Spain",["2004/2005","2020/2021"]),("Europe",["2004/2005","2020/2021"])],
    "Paris Saint-Germain": [("France",["2021/2022","2022/2023"]),("Europe",["2021/2022","2022/2023"])],
    "Inter Miami":[("United States of America",["2023","2026"]),("North and Central America",["2023","2026"])],
    "Argentina":[("International",["2004","2026"])]
}
MESSI_PLAYER_ID = 5503

In [28]:
messi_goals =[]

for team,seasons in MESSI_TEAMS.items():
    for (country,[start,end]) in seasons:
        start_ = int(start.split("/")[0])
        end_ = int(end.split("/")[0])
        comps = competitions[(competitions.country_name==country) & competitions.season_start.between(start_,end_,inclusive="both")].sort_values("season_start")
        for i,c in comps.iterrows():
            all_matches = sb.matches(competition_id=c.competition_id,season_id=c.season_id).loc[lambda x:
                    (x.home_team==team) | (x.away_team==team)].sort_values("match_date").reset_index(drop=True)
            
            for i_m,m in all_matches.iterrows():
                match_events = sb.events(match_id=m.match_id).loc[lambda x: (x.shot_outcome=="Goal") & (x.player_id==MESSI_PLAYER_ID)]
                
                if match_events.shape[0]>0:
                    for ig,g in match_events.iterrows():
                        messi_goals.append([c.competition_id,c.competition_name,c.country_name,c.season_id,c.season_name,g.team
                                            ,m.match_id,m.match_date,m.kick_off,m.stadium,
                                            g.id,g.period,g.minute,g.second,g.timestamp])

c:\Users\L781054\AppData\Local\anaconda3\envs\statsbomb\Lib\site-packages\statsbombpy\api_client.py:21: NoAuthWarning: credentials were not supplied. open data access only
  warnings.warn(
c:\Users\L781054\AppData\Local\anaconda3\envs\statsbomb\Lib\site-packages\statsbombpy\api_client.py:21: NoAuthWarning: credentials were not supplied. open data access only
  warnings.warn(
c:\Users\L781054\AppData\Local\anaconda3\envs\statsbomb\Lib\site-packages\statsbombpy\api_client.py:21: NoAuthWarning: credentials were not supplied. open data access only
  warnings.warn(
c:\Users\L781054\AppData\Local\anaconda3\envs\statsbomb\Lib\site-packages\statsbombpy\api_client.py:21: NoAuthWarning: credentials were not supplied. open data access only
  warnings.warn(
c:\Users\L781054\AppData\Local\anaconda3\envs\statsbomb\Lib\site-packages\statsbombpy\api_client.py:21: NoAuthWarning: credentials were not supplied. open data access only
  warnings.warn(
c:\Users\L781054\AppData\Local\anaconda3\envs\statsbomb

In [29]:
len(messi_goals)

509

In [30]:
df_messi_goals = pd.DataFrame.from_records(messi_goals,columns=
                                           ['competition_id','competition_name','competition_country_name','season_id','season_name','team_name',
                                            'match_id','match_date','kick_off',
                                            'stadium','id','period','minute','second','timestamp'])

In [37]:
df_messi_goals.to_csv("data/messi_goals.csv",index=False)

### Getting the kick-off time in UTC

In [38]:
df_messi_goals = pd.read_csv("data/messi_goals.csv")
print(df_messi_goals.shape)
df_messi_goals.head()

(509, 15)


,competition_id,competition_name,competition_country_name,season_id,season_name,team_name,match_id,match_date,kick_off,stadium,id,period,minute,second,timestamp
0,11,La Liga,Spain,37,2004/2005,Barcelona,68316,2005-05-01,19:00:00.000,Spotify Camp Nou,33475e03-b72c-436b-b32e-20cfaacffa74,2,90,14,00:45:14.961
1,11,La Liga,Spain,38,2005/2006,Barcelona,68354,2005-11-27,21:00:00.000,Spotify Camp Nou,e4aa0844-8e66-48fd-ab1a-06eeb71367b6,2,51,16,00:06:16.941
2,11,La Liga,Spain,38,2005/2006,Barcelona,68342,2006-01-15,19:00:00.000,Spotify Camp Nou,c024ea4d-691a-4068-809d-4472ad78db7d,2,50,45,00:05:45.673
3,11,La Liga,Spain,38,2005/2006,Barcelona,68324,2006-01-22,19:00:00.000,Spotify Camp Nou,788c4fdd-fa27-48d5-b499-3cf251b2d9fe,2,81,49,00:36:49.631
4,11,La Liga,Spain,38,2005/2006,Barcelona,68325,2006-01-29,19:00:00.000,Estadi Mallorca Son Moix,23fa10c0-c180-4c23-b2a0-67691501da4b,2,75,27,00:30:27.493


In [39]:
geolocator = Nominatim(user_agent="match_timezone_locator")
tf = TimezoneFinder()

found_timezones = {}

def find_time_zone(v):
    if v in found_timezones.keys():
        return found_timezones[v]
    location = geolocator.geocode(v)
    if location:
        tz = tf.timezone_at(lng=location.longitude, lat=location.latitude)
        found_timezones[v] = tz
        return tz
    return None

find_time_zone(df_messi_goals.iloc[0].stadium)

'Europe/Madrid'

In [40]:
df_messi_goals['timezone'] = df_messi_goals.stadium.map(find_time_zone)

In [41]:
df_messi_goals.timezone.value_counts(dropna=False)

timezone
Europe/Madrid       430
None                 41
Europe/Paris         22
Asia/Qatar            9
Atlantic/Canary       4
Europe/London         1
America/New_York      1
Europe/Moscow         1
Name: count, dtype: int64

In [42]:
df_messi_goals[df_messi_goals.timezone.isna()].stadium.value_counts()

stadium
Estadio de Mestalla                                          11
Estadio Vicente Calderón                                      8
Estadio Municipal de Ipurúa                                   8
Power Horse Stadium – Estadio de los Juegos Mediterráneos     6
Estadio Municipal El Molinón                                  4
Estadio Municipal José Zorrilla                               3
\tEstádio Cívitas Metropolitano                               1
Name: count, dtype: int64

In [43]:
df_messi_goals[df_messi_goals.timezone.isna()].competition_country_name.value_counts()

competition_country_name
Spain    41
Name: count, dtype: int64

In [44]:
df_messi_goals.loc[df_messi_goals.timezone.isna(),'timezone'] = df_messi_goals.loc[df_messi_goals.timezone.isna(),'competition_country_name'].map(find_time_zone)

In [45]:
assert df_messi_goals.timezone.isna().sum()==0

In [46]:
df_messi_goals.head(2)

,competition_id,competition_name,competition_country_name,season_id,season_name,team_name,match_id,match_date,kick_off,stadium,id,period,minute,second,timestamp,timezone
0,11,La Liga,Spain,37,2004/2005,Barcelona,68316,2005-05-01,19:00:00.000,Spotify Camp Nou,33475e03-b72c-436b-b32e-20cfaacffa74,2,90,14,00:45:14.961,Europe/Madrid
1,11,La Liga,Spain,38,2005/2006,Barcelona,68354,2005-11-27,21:00:00.000,Spotify Camp Nou,e4aa0844-8e66-48fd-ab1a-06eeb71367b6,2,51,16,00:06:16.941,Europe/Madrid


In [47]:

def local_to_utc(record):
    match_date, kick_off, timezone = record.match_date,record.kick_off,record.timezone
    local_time_str = f"{match_date} {kick_off}"
    local_time = datetime.strptime(local_time_str, "%Y-%m-%d %H:%M:%S.%f")
    local_time = pytz.timezone(timezone).localize(local_time)
    return local_time.astimezone(pytz.utc)

local_to_utc(df_messi_goals.iloc[0])

datetime.datetime(2005, 5, 1, 17, 0, tzinfo=<UTC>)

In [48]:
df_messi_goals['datetime'] = df_messi_goals.apply(local_to_utc,axis=1)

In [49]:
df_messi_goals.head()

,competition_id,competition_name,competition_country_name,season_id,season_name,team_name,match_id,match_date,kick_off,stadium,id,period,minute,second,timestamp,timezone,datetime
0,11,La Liga,Spain,37,2004/2005,Barcelona,68316,2005-05-01,19:00:00.000,Spotify Camp Nou,33475e03-b72c-436b-b32e-20cfaacffa74,2,90,14,00:45:14.961,Europe/Madrid,2005-05-01 17:00:00+00:00
1,11,La Liga,Spain,38,2005/2006,Barcelona,68354,2005-11-27,21:00:00.000,Spotify Camp Nou,e4aa0844-8e66-48fd-ab1a-06eeb71367b6,2,51,16,00:06:16.941,Europe/Madrid,2005-11-27 20:00:00+00:00
2,11,La Liga,Spain,38,2005/2006,Barcelona,68342,2006-01-15,19:00:00.000,Spotify Camp Nou,c024ea4d-691a-4068-809d-4472ad78db7d,2,50,45,00:05:45.673,Europe/Madrid,2006-01-15 18:00:00+00:00
3,11,La Liga,Spain,38,2005/2006,Barcelona,68324,2006-01-22,19:00:00.000,Spotify Camp Nou,788c4fdd-fa27-48d5-b499-3cf251b2d9fe,2,81,49,00:36:49.631,Europe/Madrid,2006-01-22 18:00:00+00:00
4,11,La Liga,Spain,38,2005/2006,Barcelona,68325,2006-01-29,19:00:00.000,Estadi Mallorca Son Moix,23fa10c0-c180-4c23-b2a0-67691501da4b,2,75,27,00:30:27.493,Europe/Madrid,2006-01-29 18:00:00+00:00


In [50]:
df_messi_goals.datetime.iloc[0].tz

<UTC>

In [51]:
WAITING_TIME_BY_PERIOD = [0,15,5,1,2]

In [52]:
pd.Series([1,2,3,4,5],index=[1,2,3,4,5]).map(lambda x: sum(WAITING_TIME_BY_PERIOD[:x]))

1     0
2    15
3    20
4    21
5    23
dtype: int64

In [53]:
df_messi_goals['goal_datetime'] = df_messi_goals.datetime + pd.to_timedelta(df_messi_goals.minute,unit="m") + pd.to_timedelta(df_messi_goals.second,unit="s") + pd.to_timedelta(df_messi_goals.period.map(lambda x: sum(WAITING_TIME_BY_PERIOD[:x])),unit="m")

In [54]:
df_messi_goals.sample(5)

,competition_id,competition_name,competition_country_name,season_id,season_name,team_name,match_id,match_date,kick_off,stadium,id,period,minute,second,timestamp,timezone,datetime,goal_datetime
150,11,La Liga,Spain,23,2011/2012,Barcelona,70225,2012-03-20,21:00:00.000,Spotify Camp Nou,0c183579-79f5-412c-8514-7b9d7a2c2851,1,16,16,00:16:16.637,Europe/Madrid,2012-03-20 20:00:00+00:00,2012-03-20 20:16:16+00:00
127,11,La Liga,Spain,23,2011/2012,Barcelona,69293,2011-10-15,20:00:00.000,Spotify Camp Nou,f6303dc6-bfe8-4cc8-9bce-7be8c551fca1,1,10,16,00:10:16.568,Europe/Madrid,2011-10-15 18:00:00+00:00,2011-10-15 18:10:16+00:00
60,11,La Liga,Spain,21,2009/2010,Barcelona,69250,2009-11-07,20:00:00.000,Spotify Camp Nou,4e048f1d-9be7-4aea-9447-adbb735f7cf9,2,86,57,00:41:57.074,Europe/Madrid,2009-11-07 19:00:00+00:00,2009-11-07 20:41:57+00:00
448,11,La Liga,Spain,90,2020/2021,Barcelona,3773660,2020-12-13,21:00:00.000,Spotify Camp Nou,e7e074ff-307a-430a-a7b2-ead97550f219,2,75,30,00:30:30.876,Europe/Madrid,2020-12-13 20:00:00+00:00,2020-12-13 21:30:30+00:00
43,11,La Liga,Spain,41,2008/2009,Barcelona,69189,2009-01-24,22:00:00.000,Spotify Camp Nou,630832e0-7961-463b-8b3a-58e6b6e63e4d,2,48,51,00:03:51.041,Europe/Madrid,2009-01-24 21:00:00+00:00,2009-01-24 22:03:51+00:00


In [57]:
# df_messi_goals['goal_datetime'] = pd.to_datetime(df_messi_goals.goal_datetime).apply(lambda x: x.replace(year=today.year)).dt.floor("min")
# df_messi_goals['goal_datetime'] = pd.to_datetime(df_messi_goals.goal_datetime)
df_messi_goals['goal_datetime_new'] = df_messi_goals['goal_datetime'].apply(lambda x: x.replace(year=datetime.now().year))
df_messi_goals['time'] = df_messi_goals.goal_datetime.dt.time
# df_messi_goals['goal_datetime_new'] = df_messi_goals['goal_datetime_new'].dt.floor("min")
df_messi_goals.sort_values(['goal_datetime_new','time'],inplace=True)
df_messi_goals['prev_goal_datetime'] = df_messi_goals.goal_datetime_new.shift(1).fillna(df_messi_goals.goal_datetime_new.iloc[-1].replace(year=datetime.now().year-1))
df_messi_goals['wait_sec'] = (df_messi_goals['goal_datetime_new'] - df_messi_goals['prev_goal_datetime']).dt.total_seconds()

In [58]:
df_messi_goals.head()

,competition_id,competition_name,competition_country_name,season_id,season_name,team_name,match_id,match_date,kick_off,stadium,...,minute,second,timestamp,timezone,datetime,goal_datetime,goal_datetime_new,time,prev_goal_datetime,wait_sec
195,11,La Liga,Spain,24,2012/2013,Barcelona,266433,2013-01-06,19:00:00.000,Spotify Camp Nou,...,28,30,00:28:30.017,Europe/Madrid,2013-01-06 18:00:00+00:00,2013-01-06 18:28:30+00:00,2025-01-06 18:28:30+00:00,18:28:30,2024-12-30 20:02:54+00:00,599136.0
398,11,La Liga,Spain,4,2018/2019,Barcelona,16120,2019-01-06,20:45:00.000,Coliseum Alfonso Pérez,...,19,50,00:19:50.307,Europe/Madrid,2019-01-06 19:45:00+00:00,2019-01-06 20:04:50+00:00,2025-01-06 20:04:50+00:00,20:04:50,2025-01-06 18:28:30+00:00,5780.0
451,11,La Liga,Spain,90,2020/2021,Barcelona,3764661,2021-01-06,21:00:00.000,San Mamés Barria,...,37,36,00:37:36.001,Europe/Madrid,2021-01-06 20:00:00+00:00,2021-01-06 20:37:36+00:00,2025-01-06 20:37:36+00:00,20:37:36,2025-01-06 20:04:50+00:00,1966.0
452,11,La Liga,Spain,90,2020/2021,Barcelona,3764661,2021-01-06,21:00:00.000,San Mamés Barria,...,61,8,00:16:08.199,Europe/Madrid,2021-01-06 20:00:00+00:00,2021-01-06 21:16:08+00:00,2025-01-06 21:16:08+00:00,21:16:08,2025-01-06 20:37:36+00:00,2312.0
364,11,La Liga,Spain,1,2017/2018,Barcelona,9742,2018-01-07,16:15:00.000,Spotify Camp Nou,...,11,35,00:11:35.890,Europe/Madrid,2018-01-07 15:15:00+00:00,2018-01-07 15:26:35+00:00,2025-01-07 15:26:35+00:00,15:26:35,2025-01-06 21:16:08+00:00,65427.0


In [64]:
df_messi_goals.drop(['goal_datetime_new','prev_goal_datetime'],axis=1).to_csv("data/messi_goals_with_goal_datetime.csv",index=False)